In [ ]:
#| default_exp oauth

# OAuth

- eval: false
- skip_exec: true

- **NB**: *This is an incomplete draft of an OAuth framework for FastHTML. It's not working yet.*

In [ ]:
#| export
import json, uuid
from httpx import get,post

from fastcore.utils import *
from fastcore.net import recv_once
from fastcore.xtras import parse_env

from oauthlib.oauth2 import WebApplicationClient
from urllib.parse import urlparse, parse_qs

In [ ]:
from IPython.display import Markdown

In [ ]:
ex_path = Path(f'../examples')
# secrets = (ex_path/'goog_secret.json').read_json()
secrets = parse_env(fn=ex_path/'gh_secret.env')
host,port = 'localhost',8000
redirect_uri = f"http://{host}:{port}/redirect"

In [ ]:
#| export
class _AppClient(WebApplicationClient):
    def __init__(self, client_id, client_secret, redirect_uri, code=None, scope=None, **kwargs):
        super().__init__(client_id, code=code, scope=scope, **kwargs)
        self.client_secret,self.redirect_uri = client_secret,redirect_uri

In [ ]:
#| export
class GoogleAppClient(_AppClient):
    "A `WebApplicationClient` for Google oauth2"
    pre = "https://www.googleapis.com/oauth2"
    acct_pre = "https://accounts.google.com/o/oauth2"
    scope_pre = "https://www.googleapis.com/auth/userinfo"

    base_url = f"{acct_pre}/v2/auth"
    token_url = f"{pre}/v4/token"
    info_url = f"{pre}/v3/userinfo"
    id_key = 'sub'
    
    def __init__(self, client_id, client_secret, redirect_uri=None, redirect_uris=None, code=None, scope=None, **kwargs):
        if redirect_uris and not redirect_uri: redirect_uri = redirect_uris[0]
        if not scope: scope=["openid", f"{self.scope_pre}.email", f"{self.scope_pre}.profile"]
        super().__init__(client_id, client_secret, redirect_uri, code=code, scope=scope, **kwargs)

In [ ]:
#| export
class GitHubAppClient(_AppClient):
    "A `WebApplicationClient` for GitHub oauth2"
    pre = "https://github.com/login/oauth"
    base_url = f"{pre}/authorize"
    token_url = f"{pre}/access_token"
    info_url = "https://api.github.com/user"
    id_key = 'id'

    def __init__(self, client_id, client_secret, redirect_uri, code=None, scope=None, **kwargs):
        if not scope: scope="user"
        super().__init__(client_id, client_secret, redirect_uri, code=code, scope=scope, **kwargs)

In [ ]:
#| export
@patch
def login_link(self:WebApplicationClient, scope=None):
    "Get a login link for this client"
    if not scope: scope=self.scope
    return self.prepare_request_uri(self.base_url, self.redirect_uri, scope)

In [ ]:
# client = GoogleAppClient(**secrets['web'], redirect_uri=redirect_uri)
client = GitHubAppClient(**secrets, redirect_uri=redirect_uri)
login_link = client.login_link()

In [ ]:
t = recv_once(host, port)
Markdown(f'[login]({login_link})')

[login](https://github.com/login/oauth/authorize?response_type=code&client_id=Ov23liKzJ7qYxQdDDtHx&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fredirect&scope=read%3Auser%2Cuser%3Aemail)

In [ ]:
#| export
def retr_code(ret):
    "Parse a request to get the code -- mainly used for testing"
    url = ret.decode()
    if url.startswith('GET'): url = url.split(' ')[1]
    query_params = parse_qs(urlparse(url).query)
    return query_params.get('code', [None])[0]

In [ ]:
code = retr_code(t.result)
code

'e0b01caa1aa03fa8cfa7'

In [ ]:
#| export
@patch
def parse_response(self:_AppClient, code):
    "Get the token from the oauth2 server response"
    payload = dict(code=code, redirect_uri=self.redirect_uri, client_id=self.client_id,
                   client_secret=self.client_secret, grant_type='authorization_code')
    self.parse_request_body_response(post(self.token_url, json=payload).text)

In [ ]:
#| export
@patch
def get_info(self:_AppClient):
    "Get the info for authenticated user"
    headers = {'Authorization': f'Bearer {self.token["access_token"]}'}
    return get(self.info_url, headers=headers).json()

In [ ]:
#| export
@patch
def retr_info(self:_AppClient, code):
    "Combines `parse_response` and `get_info`"
    self.parse_response(code)
    return self.get_info()

In [ ]:
#| export
@patch
def retr_id(self:_AppClient, code):
    "Call `retr_info` and then return id/subscriber value"
    return self.retr_info()[self.id_key]

In [ ]:
client.retr_id(code)

346999

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()